<a href="https://colab.research.google.com/github/wenny380/NlP/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install transformers
!pip install accelerate
!pip install transformers
!pip install datasets
!pip install rouge
!pip install -q evaluate rouge_score
!pip install bert-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

## Part 1: Finetuning

For your projects, you are much more likely to want to finetune a pretrained model. This is a little bit more involved, but is still quite easy.

### 2.1 Loading in a dataset

In addition to having models, the [the hub](https://huggingface.co/datasets) also has datasets.

In [62]:
from datasets import load_dataset, DatasetDict

#DataLoader(zip(list1, list2))

eurLexSum_dataset = load_dataset("lighteval/legal_summarization","EurLexSum")




In [63]:
def truncate(example):
    return {
        'article': " ".join(example['article'].split()[:200]),
        'summary': " ".join(example['summary'].split()[:100])
    }


# Take 128 random examples for train and 32 validation
eurLexSum_dataset_2 = DatasetDict(
    train=eurLexSum_dataset['train'].shuffle(seed=1111).select(range(128)),
    test=eurLexSum_dataset['train'].shuffle(seed=1111).select(range(128, 160)),
)

In [64]:
eurLexSum_dataset_2


DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 128
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 32
    })
})

In [65]:
eurLexSum_dataset_2 ['train'][:2]

{'article': ["7.12.2017 EN Official Journal of the European Union L 322/3 AGREEMENT between the European Union and the Swiss Confederation on the linking of their greenhouse gas emissions trading systems THE EUROPEAN UNION, (hereinafter referred to as ‘the Union’) of the one part, and THE SWISS CONFEDERATION, (hereinafter referred to as ‘Switzerland’) of the other part, (hereinafter referred to as ‘the Parties’), CONSCIOUS OF the global challenge presented by climate change and the international efforts required to reduce greenhouse gas emissions to combat climate change; NOTING the international commitments, in particular the United Nations Framework Convention on Climate Change (UNFCCC) and its Kyoto Protocol, to reduce greenhouse gas emissions; CONSIDERING THAT Switzerland and the Union share the objective of reducing greenhouse gas emissions significantly by 2020 and beyond; CONSCIOUS THAT revisions for future trading periods of the Emissions Trading Systems of the Union and Switze

In [66]:
def preprocess_function(examples):
  #add summarize so that the t5-small model know the task
    #max_input_length = max(len(tokenizer.encode(example['article'])) for example in examples)
    #max_target_length = max(len(tokenizer.encode(example['summary'])) for example in examples)

    inputs = ["summarize: " + text for text in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True, padding="max_length")

    # Assigns the tokenized labels to the 'labels' field of model_inputs.
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = labels["input_ids"]

    # Returns the prepared inputs and labels as a single dictionary, ready for training.
    return model_inputs

In [67]:
from transformers import AutoTokenizer, T5Model
tokenizer = AutoTokenizer.from_pretrained("t5-small")

"""inputs = ["summarize: " + text for text in example["article"]]
tokenized_billsum = billsum_dataset_2.map(
    lambda example: tokenizer(inputs, padding=true, truncation=True),
    batched=True,
    batch_size=16
)"""

tokenized_eurLexSum = eurLexSum_dataset_2.map(preprocess_function, batched=True)
#tokenized_billsum = tokenized_billsum.remove_columns(["article"])
#tokenized_billsum = tokenized_billsum.remove_columns(["summary"])
tokenized_eurLexSum.set_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [68]:
tokenized_eurLexSum['train'][0:2]

{'article': ["7.12.2017 EN Official Journal of the European Union L 322/3 AGREEMENT between the European Union and the Swiss Confederation on the linking of their greenhouse gas emissions trading systems THE EUROPEAN UNION, (hereinafter referred to as ‘the Union’) of the one part, and THE SWISS CONFEDERATION, (hereinafter referred to as ‘Switzerland’) of the other part, (hereinafter referred to as ‘the Parties’), CONSCIOUS OF the global challenge presented by climate change and the international efforts required to reduce greenhouse gas emissions to combat climate change; NOTING the international commitments, in particular the United Nations Framework Convention on Climate Change (UNFCCC) and its Kyoto Protocol, to reduce greenhouse gas emissions; CONSIDERING THAT Switzerland and the Union share the objective of reducing greenhouse gas emissions significantly by 2020 and beyond; CONSCIOUS THAT revisions for future trading periods of the Emissions Trading Systems of the Union and Switze

In [69]:
tokenized_eurLexSum['test'][0]['summary']

"European Partnership with Bosnia and Herzegovina European Partnership with Bosnia and Herzegovina The European Partnership – an instrument of the Stabilisation and Association Process – is intended to provide additional, tailored support to the Government of Bosnia and Herzegovina in order to realise the country’s European perspective. Its aim is to identify priority areas where further efforts and reforms are required, calling in particular for Bosnia and Herzegovina to bring its legislation into line with that of the Community. It also provides a reference framework for Community aid. ACT Council Decision 2008/211/EC of 18 February 2008 on the principles, priorities and conditions contained in the European Partnership with Bosnia and Herzegovina and repealing Decision 2006/55/EC. SUMMARY The European Partnership with Bosnia and Herzegovina is the main instrument for assisting the authorities of Bosnia and Herzegovina in realising their country’s European perspective, confirmed at th

In [70]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [71]:
import evaluate

rouge = evaluate.load("rouge")

In [72]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_pred):

    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [73]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

arguments = Seq2SeqTrainingArguments(
    output_dir="fine_tuned_t5small",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    #save_strategy="epoch",
    save_total_limit=3,
    learning_rate=2e-5,
    predict_with_generate=True,
    fp16=True
)


trainer = Seq2SeqTrainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_eurLexSum['train'],
    eval_dataset=tokenized_eurLexSum['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



In [74]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,7.736926,0.149200,0.066000,0.131800,0.131000,19.000000
2,No log,7.515732,0.150300,0.066800,0.132000,0.131000,19.000000
3,No log,7.384882,0.149800,0.066900,0.132100,0.131100,19.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=24, training_loss=7.627909978230794, metrics={'train_runtime': 26.8198, 'train_samples_per_second': 14.318, 'train_steps_per_second': 0.895, 'total_flos': 103942503530496.0, 'train_loss': 7.627909978230794, 'epoch': 3.0})

In [75]:
trainer.save_model("fine_tuned_t5small")

In [59]:
text = eurLexSum_dataset_2 ['test'][30]["article"]
text = "summarize: "+ text

In [76]:
from transformers import pipeline

#testing the finetune model
summarizer = pipeline("summarization", model="fine_tuned_t5small")
pred = summarizer(text)
pred


Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'federal agencies should not acquire private land, other than by exchange, unless the acquisition is clearly in the national interest . it is the policy of Congress that a federal agency should minimize the impact of federal acquisition on units of local government . if for any reason the head of an agency fails to make a payment required under subsection (a), the agency may file civil action against the agency .'}]